In [34]:
import time
from copy import deepcopy
from hashlib import sha256

## Hash

Create hash digest from string, int or float.

In [35]:
sha = lambda x: sha256(str(x).encode()).hexdigest(); sha('satoshi')

'da2876b3eb31edb4436fa4650673fc6f01f90de2f1793c4ec332b2387b09726f'

Create random hash for testing.

In [36]:
rh = lambda: sha(time.time())
h  = rh(); h

'2f082f3f8990169091a053f11749782778a98449c87e5eebaf551a334bdce31a'

Turn hash into one of 256 possible emojis. 

In [37]:
def hash2emoji(h):
    if h[:2]!='0x': h='0x'+h
    offset  = int(h[0:4], 0)
    unicode = b'\U' + b'000' + str(hex(0x1F466+offset))[2:].encode()
    return unicode.decode('unicode_escape')

hash2emoji(h)

'💕'

Print hash in a nice way.

In [38]:
def ph(h):
    if len(h)<16: return h
    else        : return hash2emoji(h)+' '+h[:16] + '...' + h[-3:]

ph(h)

'💕 2f082f3f89901690...31a'

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/03_block.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

In [47]:
class TX: 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop!='sig': super().__setattr__('hash', sha(self.__dict__))
        
    def __str__ (self): 
        is_signed = '\nsigned:\ttrue' if hasattr(self, 'sig') else '\nsigned:\tfalse'
        return ('time:\t'   + time.ctime(self.time)+
                '\nfrom:\t' + ph(self.fr)+
                '\nto:\t'   + ph(self.to)+
                '\nvalue:\t'+ str(self.value)+' ether'
                '\nfee:\t'  + str(self.fee)  +' ether' 
                '\nnonce:\t'+ str(self.nonce)+
                '\nhash:\t' + ph(self.hash)+
                is_signed)
    
    def __bytes__(self):     return str(self.hash).encode()
    def __eq__(self, other): return self.hash == other.hash

In [40]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0)

In [41]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])

print(txs2str([tx1,tx2]))

time:	Sat Mar 27 00:32:33 2021
from:	💰 4aeaf9d8386c2db5...34d
to:	📬 8668b2493f0fd4e5...843
value:	12 ether
fee:	0.2 ether
nonce:	0
hash:	🕚 f43be16d9db4a962...970
signed:	false

time:	Sat Mar 27 00:32:33 2021
from:	🔱 cb3cc6e00a171703...6f0
to:	🔥 bf9abd065ee76ef1...fe2
value:	6 ether
fee:	0.15 ether
nonce:	0
hash:	🔤 be31011042683739...f24
signed:	false



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [44]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2)

time:	Sat Mar 27 00:32:33 2021
from:	🔱 cb3cc6e00a171703...6f0
to:	🔥 bf9abd065ee76ef1...fe2
value:	6 ether
fee:	0.15 ether
nonce:	0
hash:	🔤 be31011042683739...f24
signed:	false


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [46]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [48]:
tx1_unsigned = deepcopy(tx1)
tx1.sig      = rh(); print(tx1)

time:	Sat Mar 27 00:32:33 2021
from:	💰 4aeaf9d8386c2db5...34d
to:	📬 8668b2493f0fd4e5...843
value:	12 ether
fee:	0.2 ether
nonce:	0
hash:	🕚 f43be16d9db4a962...970
signed:	true


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [50]:
assert tx1 == tx1_unsigned